### THƯ VIỆN SỬ DỤNG

In [ ]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import load_img
from tqdm.notebook import tqdm
import ast

### RÚT TRÍCH CÁC ĐẶC TRƯNG CỦA ẢNH

In [2]:
def extract_features(images, bboxs, indexs):
  features = []
  for row, image in tqdm(enumerate(images), total=len(images)):
    img = load_img(image, grayscale=True)
    bbox = bboxs[indexs[row]]
    bbox = ast.literal_eval(bbox)
    img = img.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
    img = img.resize((64, 64))
    img = np.array(img)
    features.append(img)

  features = np.array(features)
  # Ignore this stop if using RGB
  features = features.reshape(len(features), 64, 64, 1)
  return features

### TẢI MÔ HÌNH

In [3]:
def get_gender(prob):
    if prob < 0.5:return "Male"
    else: return "Female"

In [4]:
from keras.models import load_model

# Tải lại mô hình từ tệp tin 'data.h5'
model_gender = load_model('model_gender.h5')

### ĐƯA RA DỰ ĐOÁN VÀ XUẤT FILE

In [5]:
path_to_bbox = 'D:/Contest/AIFace/bbox_result.csv'
path_to_test_data = 'D:/Contest/AIFace/public_test/'
df_output = pd.read_csv(path_to_bbox)
image_paths = df_output['file_name'].tolist()
X_output = extract_features(path_to_test_data + df_output['file_name'], df_output['bbox'], df_output.index)
X_output = X_output/255.0

  0%|          | 0/2213 [00:00<?, ?it/s]

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [8]:
genders_output = []
for image_index in range(0, len(X_output)):  
    genders_output.append(get_gender(model_gender.predict(np.array([ X_output[image_index] ]), verbose=0)))

df_output['gender'] = genders_output

df_output = df_output[['file_name', 'bbox', 'gender']]

df_output.to_csv('gender_result.csv', index=False)